In [ ]:
pip install httpx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
import httpx
from bs4 import BeautifulSoup
import pandas as pd

def create_url(service_key, page_no, num_of_rows, year):
    base_url = 'https://apis.data.go.kr/B551014/SRVC_OD_API_VWEB_MBR_RACE_INFO/TODZ_API_VWEB_MBR_RACE_I'
    params = {
        'serviceKey': service_key,
        'pageNo': page_no,
        'numOfRows': num_of_rows,
        'resultType': 'XML',
        'stnd_yr': year
    }

    # URL 쿼리 문자열 생성
    query_string = '&'.join([f'{key}={value}' for key, value in params.items()])
    url = f'{base_url}?{query_string}'

    return url

# 사용자 정의 파라미터
service_key = ''
num_of_rows = 1000
year = 2024
total_pages = 3  # 총 11페이지

# 각 페이지의 DataFrame을 저장할 리스트
dataframes = []

# 모든 페이지에서 데이터 수집
for page in range(1, total_pages + 1):
    url = create_url(service_key, page, num_of_rows, year)
    try:
        response = httpx.get(url)
        response.raise_for_status()  # HTTP 오류 발생 시 예외를 발생시킴
        content = response.text

        # XML 파싱
        xml_obj = BeautifulSoup(content, 'lxml-xml')
        rows = xml_obj.find_all('item')

        # 첫 번째 행에서 열 이름 추출
        column_names = set()
        for row in rows:
            column_names.update(col.name for col in row.find_all())
        column_names = sorted(column_names)  # 열 이름을 정렬

        # 데이터 프레임을 위한 데이터 수집
        data = []
        for row in rows:
            row_data = {name: None for name in column_names}  # 열 이름을 기준으로 빈 딕셔너리 생성
            for col in row.find_all():
                row_data[col.name] = col.text  # 데이터 입력
            data.append(row_data)

        # 데이터 프레임 생성
        df = pd.DataFrame(data)
        dataframes.append(df)  # 리스트에 추가

    except httpx.HTTPStatusError as e:
        print(f"HTTP 오류 발생: {e}")
    except Exception as e:
        print(f"오류 발생: {e}")

# 모든 DataFrame을 하나로 결합
final_df = pd.concat(dataframes, ignore_index=True)

# 결과 출력
print(final_df)


     avg_acdnt_scr avg_rank bf_dd_recd_scr boat_avg_rank_scr  \
0             0.00     0.00           None              5.50   
1             0.00     0.00           None              4.27   
2             0.00     0.00           None              5.39   
3             0.00     0.00           None              5.60   
4             0.00     0.00           None              5.14   
...            ...      ...            ...               ...   
2995          0.17     5.57           None              8.00   
2996          0.41     4.09           None              0.00   
2997          0.23     5.34           None             10.00   
2998          0.07     6.37           None              5.33   
2999          0.09     5.93           None              0.00   

     boat_high_rank_ratio boat_no color_nm day_tcnt dptre_tm fl_tcnt  ...  \
0                    39.1     099        백        1    11:40    F0L0  ...   
1                    22.6     023        흑        1    11:40    F0L0  ...   


In [ ]:
final_df

,avg_acdnt_scr,avg_rank,bf_dd_recd_scr,boat_avg_rank_scr,boat_high_rank_ratio,boat_no,color_nm,day_tcnt,dptre_tm,fl_tcnt,...,thdd_race_no,tms_6_avg_rank_scr,tms_6_avg_scr,tms_6_avg_strt_tm,tms_6_high_3_rank_ratio,tms_6_high_rank_ratio,tms_6_win_ratio,tms_8_rank_ord_no,week_tcnt,wght
0,0.00,0.00,None,5.50,39.1,099,백,1,11:40,F0L0,...,08,5.71,4.76,0.20,64.7,35.3,17.6,3431 3241,1,59
1,0.00,0.00,None,4.27,22.6,023,흑,1,11:40,F0L0,...,09,5.29,5.11,0.33,42.9,35.7,14.3,6444 5421,1,54
2,0.00,0.00,None,5.39,37.3,093,적,1,11:40,F0L0,...,None,4.21,4.05,0.20,36.8,10.5,5.3,3614 3326,1,57
3,0.00,0.00,None,5.60,37.8,067,청,1,11:40,F0L0,...,10,5.19,4.88,0.24,43.8,31.3,25.0,6431 6514,1,58
4,0.00,0.00,None,5.14,33.9,078,황,1,11:40,F0L0,...,None,7.00,6.89,0.26,77.8,44.4,22.2,2424 3433,1,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.17,5.57,None,8.00,100.0,005,흑,1,13:58,F0L0,...,None,5.53,5.41,0.21,47.1,47.1,23.5,4152 1441,23,55
2996,0.41,4.09,None,0.00,0.0,097,적,1,13:58,F0L0,...,None,4.53,4.06,0.26,35.3,29.4,11.8,3524 1264,23,58
2997,0.23,5.34,None,10.00,100.0,007,청,1,13:58,F0L0,...,None,5.60,5.20,0.25,66.7,40.0,13.3,6225 2023,23,57
2998,0.07,6.37,None,5.33,0.0,018,황,1,13:58,F0L0,...,17,6.41,6.38,0.20,64.7,52.9,29.4,5214 5126,23,60


In [ ]:
import httpx
from bs4 import BeautifulSoup
import pandas as pd

def create_url(service_key, page_no, num_of_rows, year):
    base_url = 'https://apis.data.go.kr/B551014/SRVC_OD_API_VWEB_MBR_RACE_INFO/TODZ_API_VWEB_MBR_RACE_I'
    params = {
        'serviceKey': service_key,
        'pageNo': page_no,
        'numOfRows': num_of_rows,
        'resultType': 'XML',
        'stnd_yr': year
    }

    # URL 쿼리 문자열 생성
    query_string = '&'.join([f'{key}={value}' for key, value in params.items()])
    url = f'{base_url}?{query_string}'

    return url

# 사용자 정의 파라미터
service_key = ''
num_of_rows = 1000
year = 2024
total_pages = 12 # 총 11페이지

# 각 페이지의 DataFrame을 저장할 리스트
dataframes = []

# 모든 페이지에서 데이터 수집
for page in range(1, total_pages + 1):
    url = create_url(service_key, page, num_of_rows, year)
    try:
        response = httpx.get(url)
        response.raise_for_status()  # HTTP 오류 발생 시 예외를 발생시킴
        content = response.text

        # XML 파싱
        xml_obj = BeautifulSoup(content, 'lxml-xml')
        rows = xml_obj.find_all('item')

        # 첫 번째 행에서 열 이름 추출
        column_names = set()
        for row in rows:
            column_names.update(col.name for col in row.find_all())
        column_names = sorted(column_names)  # 열 이름을 정렬

        # 데이터 프레임을 위한 데이터 수집
        data = []
        for row in rows:
            row_data = {name: None for name in column_names}  # 열 이름을 기준으로 빈 딕셔너리 생성
            for col in row.find_all():
                row_data[col.name] = col.text  # 데이터 입력
            data.append(row_data)

        # 데이터 프레임 생성
        df = pd.DataFrame(data)
        dataframes.append(df)  # 리스트에 추가

    except httpx.HTTPStatusError as e:
        print(f"HTTP 오류 발생: {e}")
    except Exception as e:
        print(f"오류 발생: {e}")

# 모든 DataFrame을 하나로 결합
final_df = pd.concat(dataframes, ignore_index=True)

# 결과 출력
print(final_df)


     avg_acdnt_scr avg_rank     bf_dd_recd_scr boat_avg_rank_scr  \
0             0.00     0.00               None              5.50   
1             0.00     0.00               None              4.27   
2             0.00     0.00               None              5.39   
3             0.00     0.00               None              5.60   
4             0.00     0.00               None              5.14   
...            ...      ...                ...               ...   
7339          0.00     0.00   일03-1-3, 일11-2-5              6.69   
7340          0.00     0.00            일04-3-1              5.40   
7341          0.00     0.00            일15-4-2              4.67   
7342          0.00     0.00            일11-5-4              4.97   
7343          0.00     0.00            일10-5-3              5.56   

     boat_high_rank_ratio boat_no color_nm day_tcnt dptre_tm fl_tcnt  ...  \
0                    39.1     099        백        1    11:40    F0L0  ...   
1                    22.6    

In [ ]:
final_df['week_tcnt'] = final_df['week_tcnt'].astype(int)
final_df['day_tcnt'] = final_df['day_tcnt'].astype(int)

In [ ]:
df_sorted = final_df.sort_values(by=['week_tcnt', 'day_tcnt'], ascending=[True, True])

In [ ]:
# df_filtered = df_sorted [df_sorted ['week_tcnt'] != 1]

In [ ]:
df_filtered = df_sorted.reset_index(drop=True)

In [ ]:
df_filtered ['new_row_num'] = (df_filtered.index // 6) + 1

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_filtered

,avg_acdnt_scr,avg_rank,bf_dd_recd_scr,boat_avg_rank_scr,boat_high_rank_ratio,boat_no,color_nm,day_tcnt,dptre_tm,fl_tcnt,high_rate,mm_6_1_race_high_rank_ratio,mm_6_2_race_high_rank_ratio,mm_6_3_race_high_rank_ratio,mm_6_4_race_high_rank_ratio,mm_6_5_race_high_rank_ratio,mm_6_6_race_high_rank_ratio,mm_6_race_tcnt,mot_avg_rank_scr,mot_bf_racer_1_no,mot_high_3_rank_ratio,mot_high_rank_ratio,motor_no,race_no,race_reg_no,racer_age,racer_grd_cd,racer_nm,rmk2_rank,row_num,sex_cd,st_mthd_cd,stnd_yr,thdd_race_no,tms_6_avg_rank_scr,tms_6_avg_scr,tms_6_avg_strt_tm,tms_6_high_3_rank_ratio,tms_6_high_rank_ratio,tms_6_win_ratio,tms_8_rank_ord_no,week_tcnt,wght,new_row_num
0,0.00,0.00,None,5.50,39.1,099,백,1,11:40,F0L0,0.0,80.0/5,25.0/4,16.7/6,20.0/5,0.0/5,20.0/5,30,4.98,박설희/536,47.9,29.7,005,01,1,31,B1,정세혁,일반,1,남,002,2024,08,5.71,4.76,0.20,64.7,35.3,17.6,3431 3241,1,59,1
1,0.00,0.00,None,4.27,22.6,023,흑,1,11:40,F0L0,0.0,50.0/6,60.0/5,16.7/6,16.7/6,20.0/5,0.0/7,35,5.49,구본선/54,54.0,38.7,053,01,2,51,B1,김기한,일반,2,남,002,2024,09,5.29,5.11,0.33,42.9,35.7,14.3,6444 5421,1,54,1
2,0.00,0.00,None,5.39,37.3,093,적,1,11:40,F0L0,0.0,57.1/7,22.2/9,20.0/5,25.0/8,37.5/8,0.0/6,43,5.35,한성근/35,52.3,35.7,036,01,3,40,B1,기광서,일반,3,남,002,2024,None,4.21,4.05,0.20,36.8,10.5,5.3,3614 3326,1,57,1
3,0.00,0.00,None,5.60,37.8,067,청,1,11:40,F0L0,0.0,66.7/6,16.7/6,0.0/4,20.0/5,0.0/6,0.0/7,34,5.41,나종호/4643,52.8,36.2,028,01,4,55,B1,권명호,일반,4,남,002,2024,10,5.19,4.88,0.24,43.8,31.3,25.0,6431 6514,1,58,1
4,0.00,0.00,None,5.14,33.9,078,황,1,11:40,F0L0,0.0,85.7/7,80.0/5,57.1/7,40.0/10,25.0/8,16.7/6,43,4.85,손제민/422,44.8,31.3,042,01,5,43,A1,박진서,일반,5,남,002,2024,None,7.00,6.89,0.26,77.8,44.4,22.2,2424 3433,1,55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7339,0.00,7.60,일08-실격-5,5.21,28.6,027,흑,2,17:57,F0L0,60.0,100.0/1,00.0/0,00.0/0,0.0/1,0.0/1,100.0/1,4,4.27,김동민/633,45.5,21.2,022,17,2,44,A2,송효석,일반,6014,남,001,2024,10,7.67,7.50,18.07,100.0,66.7,16.7,2323 1233,35,56,1224
7340,0.00,2.40,일14-6-5,5.72,41.4,048,적,2,17:57,F0L0,20.0,0.0/1,0.0/1,00.0/0,0.0/1,100.0/1,00.0/0,4,4.79,강지환/563,42.4,36.4,068,17,3,40,B1,이상문,일반,6015,남,001,2024,08,2.67,2.67,18.26,16.7,16.7,0.0,2666 6425,35,57,1224
7341,0.00,5.00,일09-4-2,5.00,31.8,044,청,2,17:57,F0L0,50.0,00.0/0,0.0/1,100.0/1,00.0/0,0.0/1,100.0/1,4,4.32,김기한/615,41.9,22.6,020,17,4,44,A2,황이태,일반,6016,남,001,2024,07,5.33,5.33,18.26,50.0,50.0,16.7,2616 4253,35,57,1224
7342,0.00,6.80,"일05-1-1, 일13-4-6",6.29,47.6,072,황,2,17:57,F0L0,40.0,0.0/1,0.0/1,0.0/1,100.0/1,00.0/0,100.0/1,5,4.41,김창규/3552,41.4,31.0,052,17,5,36,A1,조성인,일반,6017,남,001,2024,09,7.00,6.92,17.98,83.3,50.0,16.7,4321 3222,35,54,1224


In [ ]:
df_filtered['new_row_num']

,new_row_num
0,1
1,1
2,1
3,1
4,1
...,...
7339,1224
7340,1224
7341,1224
7342,1224


In [ ]:
import httpx
from bs4 import BeautifulSoup
import pandas as pd

def create_url(service_key, page_no, num_of_rows, year):
    base_url = 'https://apis.data.go.kr/B551014/SRVC_OD_API_MBR_RACE_RESULT/TODZ_API_MBR_RACE_RESULT_I'
    params = {
        'serviceKey': service_key,
        'pageNo': page_no,
        'numOfRows': num_of_rows,
        'resultType': 'XML',
        'stnd_yr': year
    }

    # URL 쿼리 문자열 생성
    query_string = '&'.join([f'{key}={value}' for key, value in params.items()])
    url = f'{base_url}?{query_string}'

    return url

# 사용자 정의 파라미터
service_key = ''
page_no = 0
num_of_rows = 1000
year = 2024
total_pages = 11 # 총 3페이지

all_items = []

# 모든 페이지에서 데이터 수집
for page in range(1, total_pages + 1):
    url = create_url(service_key, page, num_of_rows, year)
    try:
        response = httpx.get(url)
        response.raise_for_status()  # HTTP 오류 발생 시 예외를 발생시킴
        content = response.text

        # XML 파싱
        xml_obj = BeautifulSoup(content, 'lxml-xml')
        items = xml_obj.find_all('item')
        all_items.extend(items)

    except httpx.HTTPStatusError as e:
        print(f"HTTP 오류 발생: {e}")
    except Exception as e:
        print(f"오류 발생: {e}")

# 새로운 XML 문서 생성
combined_xml = '<root>' + ''.join(str(item) for item in all_items) + '</root>'

# 통합된 XML 데이터 파싱
xml_obj = BeautifulSoup(combined_xml, 'lxml-xml')
rows = xml_obj.find_all('item')

# 첫 번째 행에서 열 이름 추출
columns_names = set()
for row in rows:
    columns_names.update(col.name for col in row.find_all())
columns_names = sorted(columns_names)  # 열 이름을 정렬

# 데이터 프레임을 위한 데이터 수집
data = []
for row in rows:
    row_data = {name: None for name in columns_names}  # 열 이름을 기준으로 빈 딕셔너리 생성
    for col in row.find_all():
        row_data[col.name] = col.text  # 데이터 입력
    data.append(row_data)

# 데이터 프레임 생성
df = pd.DataFrame(data)
print(df)

     day_tcnt pool1_val pool2_val pool3_val pool4_val pool5_val  pool6_val  \
0           1     ⑤15.5      ⑤5.9      ③4.3    ⑤-③106   ⑤-③39.1  ⑤-③-②31.5   
1           1      ③3.1      ③1.6      ①1.1    ③-①5.5    ③-①2.1   ③-①-⑥8.7   
2           1     ⑥14.8      ⑥4.3      ②1.4   ⑥-②45.4   ⑥-②18.1    ⑥-②-③30   
3           1      ①1.3        ①1      ⑤2.4    ①-⑤3.6    ①-⑤3.2  ①-⑤-⑥17.8   
4           1      ①1.5      ①1.6      ④1.4    ①-④5.2    ①-④3.6   ①-④-⑥6.8   
...       ...       ...       ...       ...       ...       ...        ...   
1219        2     ④20.6      ④4.5      ③1.3    ④-③107   ④-③10.8   ④-③-①2.7   
1220        2     ①18.3        ①7      ③1.5   ①-③45.2   ①-③18.8  ①-③-④17.6   
1221        2      ③2.2      ③1.6        ④1    ③-④4.4    ③-④1.9   ③-④-①2.1   
1222        2      ④1.8      ④1.5      ①1.2    ④-①4.4    ④-①1.9   ④-①-⑥3.6   
1223        2      ①1.3        ①1      ⑤1.7    ①-⑤2.1    ①-⑤1.4   ①-⑤-②1.5   

     race_no rank1 rank2 rank3 row_num stnd_yr week_tcnt  
0   

In [ ]:
df = df[['rank1','rank2','rank3','row_num']]

In [ ]:
df

,rank1,rank2,rank3,row_num
0,⑤박진서,③기광서,②김기한,1
1,③최광성,①이동준,⑥강영길,2
2,⑥장영태,②정인교,③최영재,3
3,①김인혜,⑤민영건,⑥전정환,4
4,①황이태,④박정아,⑥주은석,5
...,...,...,...,...
1219,④박민성,③최광성,①홍진수,1220
1220,①황만주,③전두식,④사재준,1221
1221,③김도휘,④심상철,①김현덕,1222
1222,④주은석,①장영태,⑥김지현,1223


In [ ]:
def extract_digits(x):
    if pd.isna(x):  # x가 NaN일 경우
        return ''
    return ''.join(filter(str.isdigit, str(x)))

# 각 열에 대해 숫자만 추출
df_cleaned = df.apply(lambda col: col.map(extract_digits))


In [ ]:
df_cleaned

,rank1,rank2,rank3,row_num
0,⑤,③,②,1
1,③,①,⑥,2
2,⑥,②,③,3
3,①,⑤,⑥,4
4,①,④,⑥,5
...,...,...,...,...
1219,④,③,①,1220
1220,①,③,④,1221
1221,③,④,①,1222
1222,④,①,⑥,1223


In [ ]:
mapping = {
    '①': 1,
    '②': 2,
    '③': 3,
    '④': 4,
    '⑤': 5,
    '⑥': 6
}

In [ ]:
df_cleaned  = df_cleaned .replace(mapping)

In [ ]:
df_cleaned

,rank1,rank2,rank3,row_num
0,5,3,2,1
1,3,1,6,2
2,6,2,3,3
3,1,5,6,4
4,1,4,6,5
...,...,...,...,...
1219,4,3,1,1220
1220,1,3,4,1221
1221,3,4,1,1222
1222,4,1,6,1223


In [ ]:
df_filtered = df_filtered.drop(columns = ['row_num'])

In [ ]:
df_filtered = df_filtered.rename(columns={'new_row_num': 'row_num'})

In [ ]:
df_filtered = df_filtered.rename(columns={'row_num': 'row_num'})
df_cleaned = df_cleaned.rename(columns={'row_num': 'row_num'})

# 데이터 타입 확인 및 변환
df_filtered['row_num'] = df_filtered['row_num'].astype(int)
df_cleaned['row_num'] = df_cleaned['row_num'].astype(int)

In [ ]:
result_df = pd.merge(df_filtered, df_cleaned, on='row_num', how='outer', suffixes=('_filtered', '_cleaned'))

In [ ]:
result_df

,avg_acdnt_scr,avg_rank,bf_dd_recd_scr,boat_avg_rank_scr,boat_high_rank_ratio,boat_no,color_nm,day_tcnt,dptre_tm,fl_tcnt,high_rate,mm_6_1_race_high_rank_ratio,mm_6_2_race_high_rank_ratio,mm_6_3_race_high_rank_ratio,mm_6_4_race_high_rank_ratio,mm_6_5_race_high_rank_ratio,mm_6_6_race_high_rank_ratio,mm_6_race_tcnt,mot_avg_rank_scr,mot_bf_racer_1_no,mot_high_3_rank_ratio,mot_high_rank_ratio,motor_no,race_no,race_reg_no,racer_age,racer_grd_cd,racer_nm,rmk2_rank,sex_cd,st_mthd_cd,stnd_yr,thdd_race_no,tms_6_avg_rank_scr,tms_6_avg_scr,tms_6_avg_strt_tm,tms_6_high_3_rank_ratio,tms_6_high_rank_ratio,tms_6_win_ratio,tms_8_rank_ord_no,week_tcnt,wght,row_num,rank1,rank2,rank3
0,0.00,0.00,None,5.50,39.1,099,백,1,11:40,F0L0,0.0,80.0/5,25.0/4,16.7/6,20.0/5,0.0/5,20.0/5,30,4.98,박설희/536,47.9,29.7,005,01,1,31,B1,정세혁,일반,남,002,2024,08,5.71,4.76,0.20,64.7,35.3,17.6,3431 3241,1,59,1,5,3,2
1,0.00,0.00,None,4.27,22.6,023,흑,1,11:40,F0L0,0.0,50.0/6,60.0/5,16.7/6,16.7/6,20.0/5,0.0/7,35,5.49,구본선/54,54.0,38.7,053,01,2,51,B1,김기한,일반,남,002,2024,09,5.29,5.11,0.33,42.9,35.7,14.3,6444 5421,1,54,1,5,3,2
2,0.00,0.00,None,5.39,37.3,093,적,1,11:40,F0L0,0.0,57.1/7,22.2/9,20.0/5,25.0/8,37.5/8,0.0/6,43,5.35,한성근/35,52.3,35.7,036,01,3,40,B1,기광서,일반,남,002,2024,None,4.21,4.05,0.20,36.8,10.5,5.3,3614 3326,1,57,1,5,3,2
3,0.00,0.00,None,5.60,37.8,067,청,1,11:40,F0L0,0.0,66.7/6,16.7/6,0.0/4,20.0/5,0.0/6,0.0/7,34,5.41,나종호/4643,52.8,36.2,028,01,4,55,B1,권명호,일반,남,002,2024,10,5.19,4.88,0.24,43.8,31.3,25.0,6431 6514,1,58,1,5,3,2
4,0.00,0.00,None,5.14,33.9,078,황,1,11:40,F0L0,0.0,85.7/7,80.0/5,57.1/7,40.0/10,25.0/8,16.7/6,43,4.85,손제민/422,44.8,31.3,042,01,5,43,A1,박진서,일반,남,002,2024,None,7.00,6.89,0.26,77.8,44.4,22.2,2424 3433,1,55,1,5,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7339,0.00,7.60,일08-실격-5,5.21,28.6,027,흑,2,17:57,F0L0,60.0,100.0/1,00.0/0,00.0/0,0.0/1,0.0/1,100.0/1,4,4.27,김동민/633,45.5,21.2,022,17,2,44,A2,송효석,일반,남,001,2024,10,7.67,7.50,18.07,100.0,66.7,16.7,2323 1233,35,56,1224,1,5,2
7340,0.00,2.40,일14-6-5,5.72,41.4,048,적,2,17:57,F0L0,20.0,0.0/1,0.0/1,00.0/0,0.0/1,100.0/1,00.0/0,4,4.79,강지환/563,42.4,36.4,068,17,3,40,B1,이상문,일반,남,001,2024,08,2.67,2.67,18.26,16.7,16.7,0.0,2666 6425,35,57,1224,1,5,2
7341,0.00,5.00,일09-4-2,5.00,31.8,044,청,2,17:57,F0L0,50.0,00.0/0,0.0/1,100.0/1,00.0/0,0.0/1,100.0/1,4,4.32,김기한/615,41.9,22.6,020,17,4,44,A2,황이태,일반,남,001,2024,07,5.33,5.33,18.26,50.0,50.0,16.7,2616 4253,35,57,1224,1,5,2
7342,0.00,6.80,"일05-1-1, 일13-4-6",6.29,47.6,072,황,2,17:57,F0L0,40.0,0.0/1,0.0/1,0.0/1,100.0/1,00.0/0,100.0/1,5,4.41,김창규/3552,41.4,31.0,052,17,5,36,A1,조성인,일반,남,001,2024,09,7.00,6.92,17.98,83.3,50.0,16.7,4321 3222,35,54,1224,1,5,2


In [ ]:
result_df.to_csv('/content/drive/MyDrive/경정데이터분석/경정데이터/정렬데이터/2024년 출주표 및 순위 .csv', encoding = 'utf-8-sig', index = False)